_See [Readme](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City#readme) for installation instructions_

---


# Santé Publique France : rendre les données de santé publique plus accessibles

## Contexte

Santé Publique France (SPF) souhaite mettre à disposition de ses agents des informations plus claires, lisibles et accessibles que les données brutes disponibles. Nous allons ici étudier les données Open Food Facts afin de les aider à mieux observer et comprendre quels sont les enjeux et problématiques de santé publique liés aux produits alimentaires de la grande distribution.

L'objectif est donc ici de produire des analyses graphiques pertinentes et parlantes au plus grand nombre.


## Outils utilisés

Nous allons utiliser le langage Python, et présenter ici le code, les résultats et l'analyse sous forme de [Notebook Jupyter](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html).

Nous allons aussi utiliser les bibliothèques usuelles d'exploration et analyse de données, afin d'améliorer la simplicité et la performance de notre code :
  * [NumPy](https://numpy.org/doc/stable/user/quickstart.html) et [Pandas](https://pandas.pydata.org/docs/user_guide/index.html) : effectuer des calculs scientifiques (statistiques, algèbre, ...) et manipuler des séries et tableaux de données volumineuses et complexes
  * [Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html), [Pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html), [Seaborn](https://seaborn.pydata.org/tutorial/function_overview.html) et [Plotly](https://plotly.com/python/getting-started/) : générer des graphiques lisibles, intéractifs et pertinents


In [ ]:
# Import libraries

# System libraries to import the data
import os.path 
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

# Math libraries to process the data
import numpy as np
import pandas as pd

# Graph libraries to produce graphs
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## If you use Notebook (and not JupyterLab), uncomment following lines
# import plotly.io as pio
# pio.renderers.default='notebook'


## Chargement des données et premier aperçu

Les données mises à disposition sont issues de [Open Food Facts](https://world.openfoodfacts.org/) et présentent les données sur les produits alimentaires.

Nous allons télécharger et extraire le fichier ZIP, puis effectuer une première passe afin de traiter les irrégularités du fichier, avant de charger les données et observer quelques valeurs.


### Téléchargement et extraction des données


In [ ]:
# Download ZIP and extract CSV
data_local_path = 'data/'
csv_filename = 'fr.openfoodfacts.org.products.csv'
csv_local_path = data_local_path+csv_filename

if not os.path.isfile(csv_local_path):
    zip_filename = csv_filename+'.zip'
    zip_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/'+zip_filename
    zip_local_path = data_local_path+zip_filename

    with urlopen(zip_url) as zip_response:
        with ZipFile(BytesIO(zip_response.read())) as zip_file:
            # extract all files do local data/ directory
            zip_file.extractall(data_local_path)


### Gestion des irrégularités du fichier CSV téléchargé

Le fichier contenant les données est mal formé à plusieurs endroits : des sauts de ligne sont présents dans 23 lignes à la fin de la colonne `first_packaging_code_geo`. Ces irrégularités sont facilement repérables car ce sont les seules lignes qui ne commencent pas par le code de l'article (`code`), mais par un séparateur `\t`. Nous allons donc corriger ces irrégularités en supprimant les sauts de ligne superflus, puis écrire les données propres dans un nouveau fichier CSV.


In [ ]:
clean_filename = 'fr.openfoodfacts.org.products-clean.csv'
clean_local_path = data_local_path+clean_filename

if not os.path.isfile(clean_local_path):
    with open(csv_local_path, 'r') as csv_file, open(clean_local_path, 'w') as clean_file:
        """ Deal with irregularities

            23 data points are wrongly split into two lines : 
            - lines : 189070, 189105, 189111, 189121, 189154, 189162, 189164, 189170, 189244, 189246, 
                    189250, 189252, 189262, 189264, 189271, 189274, 189347, 189364, 189366, 189381, 
                    189406, 189408, 189419
            
            The pattern is always the same : 
            - a NewLine character (`\n`) is placed at the end of column "first_packaging_code_geo" 
            - and the next line starts with a TAB separator (`\t`) : column "cities" is empty.
            
            Since the first column ("code") is never empty, we just remove any `\n` character that is 
            directly followed by a TAB separator (`\t`).
        """

        data = csv_file.read()
        clean_file.write(data.replace('\n\t', '\t'))


### Chargement des données

Nous allons charger les données en mémoire et convertir les valeurs dans le bon type, selon les [spécifications fournies](https://static.openfoodfacts.org/data/data-fields.txt).

In [ ]:
# Read column names
column_names = pd.read_csv(clean_local_path, sep='\t', encoding='utf-8', nrows=0).columns.values

# Set column types according to fields description (https://static.openfoodfacts.org/data/data-fields.txt)
column_types = {col: 'Int64' for (col) in column_names if col.endswith(('_t', '_n'))}
column_types |= {col: float for (col) in column_names if col.endswith(('_100g', '_serving'))}
column_types |= {col: str for (col) in column_names if not col.endswith(('_t', '_n', '_100g', '_serving', '_tags'))}

tags_converter = lambda list_as_string_value : list_as_string_value.split(',') if list_as_string_value else pd.NA

# Load raw data
raw_data = pd.read_csv(clean_local_path, sep='\t', encoding='utf-8',
    dtype=column_types,
    parse_dates=[col for (col) in column_names if col.endswith('_datetime')],
    infer_datetime_format=True,
    converters={
        # Convert '_tags' columns into list of values (separator : ',')
        col: tags_converter
        for (col) in column_names if col.endswith('_tags')
    }
)

# Display DataFrame size
raw_data.info()

Le fichier de données fourni contient 162 variables pour 320749 individus.


### Sélection des données pertinentes

Nous allons chercher à n'utiliser que les variables pertinentes pour SPF : celles pour lesquelles nous avons suffisament de valeurs non vides pour pouvoir faire une analyse statistique fiable, et qui peuvent avoir un réel sens du point de vue des problématiques de santé publique.


In [ ]:
def plot_empty_values(dataframe: pd.DataFrame) -> None:
    """ Plot a histogram of empty values percentage per columns of the input DataFrame
    """
    num_rows = len(dataframe.index)
    columns_emptiness = pd.DataFrame({
        col : { 
            'count': dataframe[col].isna().sum(),
            'percent': 100 * dataframe[col].isna().sum() / num_rows,
        } for col in dataframe.columns
    }).transpose().sort_values(by=['count'])

    fig = px.bar(columns_emptiness,
        color='percent',
        y='percent',
        labels={
            'index':'column name',
            'percent':'% of empty values',
            'count':'# of empty values',
        },
        hover_data=['count'],
        title='Empty values per column',
        width=1200,
        height=600,
    )
    fig.show()

plot_empty_values(raw_data)

Nous voyons que un grand nombre de variables ont un taux de complétude très faible et ne seront donc pas utilisables.
Nous allons donc restreindre notre analyse aux variables utilisées pour le calcul du [Nutri Score](https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score), qui est un indicateur très parlant du point de vue de la santé.


### Premier aperçu

Affichons quelques informations et les premières vaeulrs observées.


In [ ]:
# Let's keep only meaningful columns
meaningful_columns = [
    # General information
    'code', 'product_name', 'main_category', 'additives_n', 

    # Nutri-Score
    'nutrition_grade_fr', 'nutrition-score-fr_100g',

    # Positive nutrition facts
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',

    # Negative nutrition facts 
    'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g',
]
meaningful_data = raw_data.loc[:, meaningful_columns].copy()

# Display DataFrame size
meaningful_data.info()

# Display first values of each column
meaningful_data.head(20)


### Première analyse statistique

Voyons quelle est la répartition des différentes variables.


In [ ]:
# Display statistical summary of each column
meaningful_data.describe(include="all", datetime_is_numeric=True)


## Distribution des différentes valeurs

Voyons comment sont distribuées certaines variables.


### Variable catégorique nominale : `main_category`

Voyons comment sont réparties les catégories de produits.


In [ ]:
# Display the density of product categories
fig = px.line(raw_data['main_category'].value_counts())
fig.update_layout(
    title_text="Product categories",
    width=1200,
    height=800,
)
fig.show()

Nous voyons déjà qu'il y a une répartition très inégale des catégories de produits et qu'il faudrait certainement améliorer la catégorisation afin d'éviter d'avoir un très grand nombre de catégories à 1 seul élément.


In [ ]:
# Let's keep only the top values and merge the rest into "Other"
meaningful_data.loc[:,'top_category'] = raw_data['main_category'].where(
    raw_data['main_category'].isna() | raw_data['main_category'].isin(raw_data['main_category'].value_counts().index[:20]), 
    other='other', 
)

fig = make_subplots(
    rows=1, cols=2, 
    subplot_titles=("Top 20 with 'other'", "Top 20"), 
    specs=[[{'type':'domain'}, {'type':'domain'}]],
)
fig.add_trace(go.Pie(
    labels=meaningful_data['top_category'].value_counts().index, 
    values=meaningful_data['top_category'].value_counts().values, 
    name="Including 'other'",
    pull=[0.05 if cat == 'other' else 0 for cat in meaningful_data['top_category'].value_counts().index],
), row=1, col=1)
fig.add_trace(go.Pie(
    labels=meaningful_data['top_category'].value_counts().index[1:], 
    values=meaningful_data['top_category'].value_counts().values[1:], 
    name="Top 20",
), row=1, col=2)
fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)
fig.update_layout(
    title_text="Product categories",
    width=1200,
    height=600,
)
fig.show()


Nous voyons que les 20 catégories les plus représentées représentent près de 50% de toutes les valeurs. De même, les 5 premières catégories représentent plus de 80% des 3543 valeurs possibles.



### Variable catégorique ordinale : `nutrition_grade_fr`

Voyons comment sont réparties les notes de Nutri-Score.


In [ ]:
NUTRITION_GRADES = ('a', 'b', 'c', 'd', 'e')

# Display the nutrition grade distribution per product category
fig = px.histogram(meaningful_data.loc[meaningful_data['top_category'].notnull() & meaningful_data['nutrition_grade_fr'].notnull()],
    x='nutrition_grade_fr',
    category_orders={'nutrition_grade_fr': NUTRITION_GRADES},
    color='top_category',
    title='Global nutrition grade repartition',
    width=1200,
    height=600,
)
fig.show()


Nous voyons que parmis les produits répertoriés, la répartition des produits par Nutri-Score est globalement comparable, avec une sur-représentation des labels C et D .


In [ ]:
# Display the product category distribution by nutrition grade
fig = px.histogram(meaningful_data.loc[meaningful_data['top_category'] != 'other'].loc[meaningful_data['top_category'].notnull() & meaningful_data['nutrition_grade_fr'].notnull()],
    x='top_category',
    category_orders={'nutrition_grade_fr': NUTRITION_GRADES},
    color='nutrition_grade_fr',
    histnorm='percent',
    width=1200,
    height=600,
)
fig.show()


In [ ]:
# Display the heatmap of top product categories per nutrition grade
table = meaningful_data.loc[meaningful_data['top_category'] != 'other'].pivot_table(
    values='code',
    index='top_category',
    columns='nutrition_grade_fr',
    aggfunc='count',
    fill_value=0,
    observed=True,
).sort_values(by=list(NUTRITION_GRADES))

fig = px.imshow(table,
    title="Nutri-Score grade for top 20 product categories",
    width=1200,
    height=800,
)
fig.show()


Nous voyons déjàa que parmis les 20 types de produits les plus représentés, ceux présentant un meilleur Nutri-Score sont les produits en boîte, les produits à base de plantes et les pâtes. Les légumes frais sont le plus souvent de Nutri-Score A, tandis que les chocolats, bonbons, biscuits et en-cas sucrés ont le plus souvent un mauvais Nutri-Score.

### Variables numérique

Nous allons dans un premier temps nettoyer les données numérique, avant d'en étudier la répartition. Ceci permettra d'avoir des données plus fiables.


#### Nettoyage : suppression des données aberrantes

Parmis les données fournies, nous voyons des valeurs négatives pour des variables comme le nombre d'additifs, ou la quantité de sucre pour 100g de produit, ce qui est impossible.
Nous allons aussi utiliser la méthode IQR (Inter Quartile Range) pour identifier les données aberrantes (outliers) et les supprimer.


In [ ]:
def remove_negative_values(dataframe: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    """ Remove negative values from specified columns of DataFrame
    """
    df = dataframe.copy()
    for col in columns:
        df.loc[:,col] = dataframe[col].where(dataframe[col] >= 0)

    return df


positive_columns = [
    'additives_n', 
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
    'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g',
]

clean_data = remove_negative_values(meaningful_data, positive_columns)

clean_data.describe()

Nous avons bien supprimé les valeurs négatives impossibles, mais nous voyons encore des valeurs maximum aberrantes (ex. : 550g d'acides gras saturés pour 100g de produit).
Nous allons donc supprimer les valeurs aberrantes restantes grâce à la méthode IQR.


In [ ]:
def remove_outliers(dataframe: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    """ Remove outlier values from specified columns of DataFrame

        Compute the Inter-Quartile Ranges and set outliers to NaN
    """
    df = dataframe.copy()

    # compute quartiles and define range
    quartiles = df[columns].quantile([0.25, 0.75])
    iqr = quartiles.loc[0.75]-quartiles.loc[0.25]
    limits = pd.DataFrame({
        col: [
            quartiles.loc[0.25, col] - 1.5 * iqr[col], # min
            quartiles.loc[0.75, col] + 1.5 * iqr[col], # max
        ] for col in columns
    }, index=['min', 'max'])

    # set to NaN data that are outside the range
    for col in columns:
        df.loc[:,col] = dataframe[col].where(
            limits.loc['min', col] <= dataframe[col]
        ).where(
            dataframe[col] <= limits.loc['max', col]
        )

    return df


numeric_columns = positive_columns.copy()
numeric_columns.append('nutrition-score-fr_100g')

clean_data = remove_outliers(clean_data, numeric_columns)

clean_data.describe()

Nous avons maintenant des valeurs qui semblent correctes. Observons leur répartition avant de compléter les valeurs vides.


In [ ]:
# Let's define a function to plot multiple BoxPlots
def draw_boxplots(dataframe: pd.DataFrame, categorical_column: str, numerical_columns: list[str], order_values: tuple[str]) -> None:
    """ Draw one boxplot per numerical variable, split per categories.

        Arguments :
        - dataframe : Pandas DataFrame containing the data, including the categorical_column and numerical_columns
        - categorical_column : string representing the name of the variable containing the categories
        - numerical_columns : list of strings representing the name of the numerical variables to plot
        - order_values : list of strings representing the values of the numerical variables to plot

        Returns : None
    """
    num_cols = 3
    num_lines = int(np.ceil(len(numerical_columns) / 3))
    fig, axes = plt.subplots(num_lines, num_cols, figsize=(8 * num_cols , 8 * num_lines))
    fig.suptitle(f'Numeric variables distribution, per { categorical_column }', fontsize=24)

    for i, col in enumerate(numerical_columns):
        sns.boxplot(data=dataframe,
            x=categorical_column, 
            y=col,
            order=order_values,
            ax=axes[int(np.floor(i / num_cols)), i % num_cols],
        )

# Draw the BoxPlots of each numeric column, split per Nutrition Grade
draw_boxplots(
    dataframe=clean_data, 
    categorical_column='nutrition_grade_fr', 
    numerical_columns=numeric_columns, 
    order_values=NUTRITION_GRADES
)


Nous voyons qu'il y a encore des données aberrantes, notamment en analysant les données selon la note de Nutri-Score.
Nous allons donc à nouveau supprimer ces données.


In [ ]:
# Let's work on a copy of our clean data
super_clean_data = clean_data.copy()

for grade in NUTRITION_GRADES:
    # for each nutrition grade, 
    # we remove the outliers of each numeric column detected 
    # after filtering the data by nutrition grade
    super_clean_data.loc[super_clean_data['nutrition_grade_fr'] == grade] = remove_outliers(clean_data.loc[clean_data['nutrition_grade_fr'] == grade], numeric_columns)

super_clean_data.describe()


In [ ]:
# Let's draw the BoxPlots again after more data cleaning
draw_boxplots(
    dataframe=super_clean_data, 
    categorical_column='nutrition_grade_fr', 
    numerical_columns=numeric_columns, 
    order_values=NUTRITION_GRADES
)


Nous voyons maintenant que les données sont quasiment toutes contenues dans les "moustaches" des boxplots, autrement dit il n'y a presque plus de données aberrantes.


#### Nettoyage : suppression des doublons et lignes vides

Il est inutile de conserver des données en doublons, nous allons donc supprimer ces lignes. En l'occurrence, comme le `code` est unique à chaque produit, il ne peut pas y avoir de lignes en doublon.


In [ ]:
# Let's count duplucated lines
duplicates = super_clean_data.duplicated()
duplicates.describe()


Par la suite, toute notre analyse va se baser sur le paramètre `nutrition_grade_fr`. Nous allons supprimer les lignes où la valeur de cette variable est vide.


In [ ]:
# Let's drop raws where nutrition_grade_fr is empty
nutrition_data = super_clean_data.dropna(subset=['nutrition_grade_fr']).copy()
plot_empty_values(nutrition_data)


#### Nettoyage : remplacement des valeurs manquantes

Nous pouvons alors considérer les données restantes comme fiables, et nous allons nous baser sur ces valeurs pour extrapoler les valeurs manquantes dans notre jeu de données.
Nous allons remplacer, pour chaque valeur numérique vide, la valeur moyenne des individus ayant la même note de Nutri-Score.


In [ ]:
means = nutrition_data.groupby('nutrition_grade_fr').mean()
means.loc[:,'additives_n'] = means['additives_n'].map(np.round)

for grade in means.index:
    nutrition_data.loc[nutrition_data['nutrition_grade_fr'] == grade] = nutrition_data[nutrition_data['nutrition_grade_fr'] == grade].fillna(
        value=means.loc[grade]
    )

plot_empty_values(nutrition_data)

In [ ]:
# Let's draw the BoxPlots again after more data cleaning
draw_boxplots(
    dataframe=nutrition_data, 
    categorical_column='nutrition_grade_fr', 
    numerical_columns=numeric_columns, 
    order_values=NUTRITION_GRADES
)

In [ ]:
fig = px.scatter_matrix(nutrition_data.sample(frac=.01),
    dimensions=[
        'energy_100g', 
        'saturated-fat_100g', 
        'sugars_100g', 
        'salt_100g', 
        'nutrition-score-fr_100g',
    ],
    color="nutrition_grade_fr", symbol="nutrition_grade_fr",
    category_orders={
        'nutrition_grade_fr': NUTRITION_GRADES,
    },
    opacity=.1,
    width=1200,
    height=1200,
)
fig.update_traces(
    showupperhalf=False,
    diagonal_visible=False,
)
fig.show()

In [ ]:
fig = px.scatter_matrix(nutrition_data.sample(frac=.01),
    dimensions=[
        'nutrition-score-fr_100g',
        'fruits-vegetables-nuts_100g', 
        'fiber_100g',
        'proteins_100g',
    ],
    color="nutrition_grade_fr", symbol="nutrition_grade_fr",
    category_orders={
        'nutrition_grade_fr': NUTRITION_GRADES,
    },
    opacity=.1,
    width=1200,
    height=1200,
)
fig.update_traces(
    showupperhalf=False,
    diagonal_visible=False,
)
fig.show()

In [ ]:
normalized_nutrition_data = nutrition_data.copy()
normalized_nutrition_data[numeric_columns]=(nutrition_data[numeric_columns]-nutrition_data[numeric_columns].min())/(nutrition_data[numeric_columns].max()-nutrition_data[numeric_columns].min())

fig = px.scatter(normalized_nutrition_data.sample(frac=.01),
    x=[
        'additives_n', 
        'energy_100g', 
        'saturated-fat_100g', 
        'sugars_100g', 
        'salt_100g', 
        # 'sodium_100g',
        # 'nutrition-score-fr_100g',
        'fruits-vegetables-nuts_100g', 
        'fiber_100g',
        'proteins_100g',
    ],
    y='nutrition-score-fr_100g',
    # color="nutrition_grade_fr", symbol="nutrition_grade_fr",
    # category_orders={'nutrition_grade_fr': NUTRITION_GRADES},
    trendline='ols',
    opacity=.1,
    width=1200,
    height=1200,
)
fig.show()

---
---
---
---
---
---
---
---
---
---

In [ ]:
tag_columns = [
    'brands_tags', 'countries_tags',
]

brands_freq = pd.Series([x for _list in raw_data['brands_tags'].dropna() for x in _list]).value_counts()
countries_freq = pd.Series([x for _list in raw_data['countries_tags'].dropna() for x in _list]).value_counts()

display(brands_freq, countries_freq)


In [ ]:

meaningful_data['coherent_grade_score'] = False

meaningful_data.loc[
    (( meaningful_data['nutrition_grade_fr'] == 'a' ) 
        & ( meaningful_data['nutrition-score-fr_100g'] < 0 ))
    | (( meaningful_data['nutrition_grade_fr'] == 'b' ) 
        & (( 0 <= meaningful_data['nutrition-score-fr_100g'] ) 
            | ( meaningful_data['nutrition-score-fr_100g'] < 3 )))
    | (( meaningful_data['nutrition_grade_fr'] == 'c' ) 
        & (( 3 <= meaningful_data['nutrition-score-fr_100g'] ) 
            | ( meaningful_data['nutrition-score-fr_100g'] < 11 )))
    | (( meaningful_data['nutrition_grade_fr'] == 'd' ) 
        & (( 11 <= meaningful_data['nutrition-score-fr_100g'] ) 
            | ( meaningful_data['nutrition-score-fr_100g'] < 19 )))
    | (( meaningful_data['nutrition_grade_fr'] == 'e' )
        & ( 19 <= meaningful_data['nutrition-score-fr_100g'] ))
, 'coherent_grade_score'] = True

meaningful_data['coherent_grade_score'].value_counts()


In [ ]:

sns.jointplot(data=meaningful_data,
    x="saturated-fat_100g", 
    y="sugars_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    height=10,
)

In [ ]:

def expected_score(row):
    negative_points = {
        'energy_100g': [335, 670, 1005, 1340, 1675, 2010, 2345, 2680, 3015, 3350],
        'saturated-fat_100g': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'sugars_100g': [4.5, 9, 13.5, 18, 22.5, 27, 31, 36, 40, 45],
        'salt_100g': [90*2.5/1000, 180*2.5/1000, 270*2.5/1000, 360*2.5/1000, 450*2.5/1000, 540*2.5/1000, 630*2.5/1000, 720*2.5/1000, 810*2.5/1000, 900*2.5/1000],
    }
    positive_points = {
        'fruits-vegetables-nuts_100g': [40, 60, 67, 74, 80],
        'fiber_100g': [.9, 1.9, 2.8, 3.7, 4.7],
        'proteins_100g': [1.6, 3.2, 4.8, 6.4, 8.0],
    }

    score = 0

    for criteria in negative_points:
        for value in negative_points[criteria]:
            if row[criteria] > value:
                score += 1
            else:
                break

    for criteria in positive_points:
        for value in positive_points[criteria]:
            if row[criteria] > value:
                score -= 1
            else:
                break

    return score


# meaningful_data.assign(expected_score = lambda row: expected_score(row), axis=1)
# meaningful_data.assign(expected_grade = lambda row: 
#     'a' if row['expected_score'] < 0 
#     else 'b' if row['expected_score'] < 3 
#     else 'c' if row['expected_score'] < 11 
#     else 'd' if row['expected_score'] < 19 
#     else 'e' 
# , axis=1)
# meaningful_data.head(20)


In [ ]:

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="energy_100g", 
    hue="nutrition_grade_fr",
    hue_order=NUTRITION_GRADES,
    ax=axes[0,0],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="saturated-fat_100g", 
    hue="nutrition_grade_fr",
    hue_order=NUTRITION_GRADES,
    ax=axes[0,1],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="sugars_100g", 
    hue="nutrition_grade_fr",
    hue_order=NUTRITION_GRADES,
    ax=axes[1,0],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="salt_100g", 
    hue="nutrition_grade_fr",
    hue_order=NUTRITION_GRADES,
    ax=axes[1,1],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="fiber_100g", 
    hue="nutrition_grade_fr",
    hue_order=NUTRITION_GRADES,
    ax=axes[2,0],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="proteins_100g", 
    hue="nutrition_grade_fr",
    hue_order=NUTRITION_GRADES,
    ax=axes[2,1],
)


In [ ]:

clean_data = raw_data.drop_duplicates()

# Display data types and empty values
clean_data.info()

clean_data.dropna(
        axis='columns',
        thresh=.1 * num_rows,
        inplace=True,
    )

# Display data types and empty values
clean_data.info()

num_cols = len(clean_data.columns)
clean_data.dropna(
        axis='index',
        thresh=.1 * num_cols,
        inplace=True,
    )

# Display data types and empty values
clean_data.info()

clean_data.drop_duplicates(inplace=True,)

# Display data types and empty values
clean_data.info()

# Display statistical summary of each column
clean_data.describe(include="all")


In [ ]:

fig = px.imshow(clean_data.isna(),
)
fig.show()


In [ ]:

meaningful_columns = ['product_name',
    'packaging_tags', 'brands_tags', 'manufacturing_places_tags', 'countries_tags',
    'main_category', 'categories_tags', 
    'labels_tags', 'additives_n', 'additives_tags', 
    'nutrition_grade_fr', 'nutrition-score-fr_100g',
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
    'fiber_100g', 'proteins_100g',
]


In [ ]:

# display value frequencies per column
for col in clean_data.columns:
    print(f'\n \
================================================\n \
>    { col }\n \
------------------------------------------------')
    display(clean_data[col].value_counts(dropna=False))


In [ ]:

raw_data = pd.read_csv(csv_local_path, 
    sep='\t',
    usecols=['product_name',
        'packaging_tags', 'brands_tags', 'manufacturing_places_tags', 'countries_tags',
        'main_category', 'categories_tags', 
        'labels_tags', 'additives_n', 'additives_tags', 
        'nutrition_grade_fr', 'nutrition-score-fr_100g',
        'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
        'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g',
    ])

# display first 5 rows
raw_data.head()


## Effectuer des opérations de nettoyage sur des données structurées



In [ ]:
# ❒ les éventuelles valeurs manquantes de chaque colonnes ont été identifiées, quantifiées et traitées



In [ ]:
# ❒ les lignes dupliquées ont été identifiées, quantifiées et traitées



In [ ]:
# ❒ au moins une fonction a été écrite, testée et utilisée pour nettoyer le jeu de données



In [ ]:
# ❒ une méthodologie de traitement des valeurs manquantes pour chaque colonne est justifiée et mise en oeuvre (ex : remplacer les valeurs manquantes d’une colonne par la valeur moyenne de la colonne)



In [ ]:
# ❒ une méthodologie de traitement des lignes dupliquées est justifiée et mise en oeuvre (ex : les lignes doublons ont été supprimés)  



In [ ]:
# ❒ les fonctionnalités d’édition de cellule Markdown du Jupyter Notebook sont utilisées dans au moins trois cellules pour décrire les choix méthodologiques et rendre lisible le document (titres, mise en forme, alternance de cellule d’exécution de code Python et de cellule de texte explicatif) 



In [ ]:
# ❒ la démarche de nettoyage des données est visible dans la structure du document (découpage du document en partie avec des titres clairs et mis en évidence, des commentaires à l’intérieur des parties pour expliciter la démarche, …)



## Effectuer une analyse statistique multivariée



In [ ]:
# ❒ au moins une méthode d’analyse descriptive est appliquée sur le jeu de données (ex : ACP)



In [ ]:
# ❒ au moins une méthode d’analyse explicative est appliquée sur le jeu de données (ex : ANOVA)



In [ ]:
# ❒ au moins une fonction a été écrite, testée et utilisée pour effectuer une analyse statistique multivariée



In [ ]:
# ❒ la méthode d’analyse descriptive appliquée sur le jeu de données est expliquée et justifiée



In [ ]:
# ❒ la méthode d’analyse explicative appliquée sur le jeu de données est expliquée et justifiée



## Communiquer ses résultats à l’aide de représentations graphiques lisibles et pertinentes



In [ ]:
# ❒ au moins trois types différents de graphiques ont été utilisés (ex : histogramme, boîte à moustache, nuage de points)



In [ ]:
# ❒ la justification des types de graphiques utilisés est explicitée dans le Jupyter Notebook.



In [ ]:
# ❒ au moins une fonction a été écrite, testée et utilisée pour effectuer une représentation graphique



In [ ]:
# ❒ les titres, valeurs des axes des abscisses et des ordonnées et légendes sont explicites



In [ ]:
# ❒ au moins un graphique interactif est utilisé pour illustrer une analyse lors de la présentation.



In [ ]:
# ❒ les titres, valeurs des axes des abscisses et des ordonnées et légendes sont affichés de manière lisible



---

_[Licence GPL-v3](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City/blob/main/LICENSE)_
